In [1]:
!pip install -q mediapipe

  You can safely remove it manually.
  You can safely remove it manually.



The next step you will take is downloading an off-the-shelf model for audio classification. In this case you will use the YAMNet model, which is designed to classify audio in 0.975 second segments, though you are also able to use others, including your own custom models, with MediaPipe Tasks.

In [2]:
!wget -O classifier.tflite -q https://storage.googleapis.com/mediapipe-models/audio_classifier/yamnet/float32/1/yamnet.tflite

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [25]:
# Install necessary libraries
!pip install pydub ffmpeg


In [26]:
!pip install ffmpeg-python


  Using cached future-1.0.0-py3-none-any.whl.metadata (4.0 kB)
Using cached future-1.0.0-py3-none-any.whl (491 kB)


## Performing Audio Classification
Now that you have the necessary dependencies, it's time to start classifying some audio! While there are a variety of ways to retrieve audio clips, this example will download a `.wav` file of someone speaking.

Once everything looks good, you can start performing inference. You will start by creating the options that are necessary for associating your model with the Audio Classifier, as well as some other customizations.

Next, you will create your Classifier and read some information from your downloaded audio file, as well as segment the clip into smaller (0.975 seconds, in this case) clips before classifying them.

Finally, you will loop through the audio file in increments of 975 (the amount of seconds per clip in millesconds) to display the classification results.

In [27]:

# Import libraries
from pydub import AudioSegment

def convert_mp3_to_wav(input_mp3_path, output_wav_path):
    """
    Converts an MP3 file to WAV format in Google Colab.

    Parameters:
        input_mp3_path (str): Path to the input MP3 file.
        output_wav_path (str): Path to save the converted WAV file.

    Returns:
        None
    """
    try:
        # Load the MP3 file
        print(f"Converting MP3 to WAV: {input_mp3_path}")
        audio = AudioSegment.from_mp3(input_mp3_path)
        # Export the audio as a WAV file
        audio.export(output_wav_path, format="wav")
        print(f"Conversion successful: {output_wav_path}")
    except Exception as e:
        print(f"An error occurred during conversion: {e}")


In [28]:
import os

def get_all_mp3_files(directory):
    """
    Get all file paths with .mp3 extension from a directory.

    Parameters:
        directory (str): Path to the directory to search.

    Returns:
        list: List of file paths with .mp3 extension.
    """
    mp3_files = []
    try:
        # Walk through the directory
        for root, dirs, files in os.walk(directory):
            for file in files:
                if file.endswith(".mp3"):
                    mp3_files.append(os.path.join(root, file))
        return mp3_files
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

# Example usage
mp3_files = get_all_mp3_files("mp3/")
print("Found MP3 files:", mp3_files)


Found MP3 files: ['mp3/applause-01-253125.mp3', 'mp3/applause-cheer-236786.mp3', 'mp3/crowd-shouting-hey-hey-hey-272059.mp3', 'mp3/dog-barking-265976.mp3', 'mp3/hungry-cat-254850.mp3', 'mp3/relaxing-guitar-loop-v5-245859.mp3', 'mp3/weird-talking-47375.mp3', 'mp3/woman-crying-softly-268484.mp3']


In [29]:
output_wav_path = "output/"  # Name for the converted WAV file
# input_mp3_paths = mp3_files

for i, path in enumerate(mp3_files):
  print(path)
  output = output_wav_path + str(i) + ".wav"
  convert_mp3_to_wav(path,output)




mp3/applause-01-253125.mp3
Converting MP3 to WAV: mp3/applause-01-253125.mp3
An error occurred during conversion: [WinError 2] The system cannot find the file specified
mp3/applause-cheer-236786.mp3
Converting MP3 to WAV: mp3/applause-cheer-236786.mp3
An error occurred during conversion: [WinError 2] The system cannot find the file specified
mp3/crowd-shouting-hey-hey-hey-272059.mp3
Converting MP3 to WAV: mp3/crowd-shouting-hey-hey-hey-272059.mp3
An error occurred during conversion: [WinError 2] The system cannot find the file specified
mp3/dog-barking-265976.mp3
Converting MP3 to WAV: mp3/dog-barking-265976.mp3
An error occurred during conversion: [WinError 2] The system cannot find the file specified
mp3/hungry-cat-254850.mp3
Converting MP3 to WAV: mp3/hungry-cat-254850.mp3
An error occurred during conversion: [WinError 2] The system cannot find the file specified
mp3/relaxing-guitar-loop-v5-245859.mp3
Converting MP3 to WAV: mp3/relaxing-guitar-loop-v5-245859.mp3
An error occurred du

In [30]:
audio_file_name = "output/output6.wav"

In [31]:
import numpy as np

from mediapipe.tasks import python
from mediapipe.tasks.python.components import containers
from mediapipe.tasks.python import audio
from scipy.io import wavfile

# Customize and associate model for Classifier
base_options = python.BaseOptions(model_asset_path='classifier.tflite')
options = audio.AudioClassifierOptions(
    base_options=base_options, max_results=4)

# Create classifier, segment audio clips, and classify
with audio.AudioClassifier.create_from_options(options) as classifier:
  sample_rate, wav_data = wavfile.read(audio_file_name)
  audio_clip = containers.AudioData.create_from_array(
      wav_data.astype(float) / np.iinfo(np.int16).max, sample_rate)
  classification_result_list = classifier.classify(audio_clip)

  # assert(len(classification_result_list) == 5)

# Iterate through clips to display classifications
  for idx, timestamp in enumerate(range(242)):
    classification_result = classification_result_list[idx]
    top_category = classification_result.classifications[0].categories[0]
    print(f'Timestamp {timestamp}: {top_category.category_name} ({top_category.score:.2f})')

ImportError: DLL load failed while importing _framework_bindings: A dynamic link library (DLL) initialization routine failed.